# Reading in a file from Google Drive

In [ ]:
# Authenticate
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("Authenticated")

Authenticated


In [ ]:
# change link to the input file
link = 'https://drive.google.com/file/d/1f1m87RaNUy6uzW76b0oNlRTMfx5mHYUa/view?usp=sharing'

# read in the file
import pandas as pd
id = link.split("/")[-2] 
  
downloaded = drive.CreateFile({'id':id})  

# change file name 
downloaded.GetContentFile('hotel_bookings_raw.csv')   
  
df = pd.read_csv('hotel_bookings_raw.csv') 


# Reading in a file from local upload


In [ ]:
# select file to upload
from google.colab import files
uploaded = files.upload()

Saving hotel_bookings_raw.csv to hotel_bookings_raw (1).csv


In [ ]:
import io
# change file name
df2 = pd.read_csv(io.BytesIO(uploaded['hotel_bookings_raw.csv']))

# Downloading file

In [ ]:
from google.colab import files

df.to_csv('df.csv')
files.download('df.csv')




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install  spotipy

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 153kB 10.8MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


SPOTIPY 

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="87b59c13e62f40fe8e88a5e8d691d4c4",
                                               client_secret="3b206b365c7d45a1a54a868153f7c1b1",
                                               redirect_uri="http://localhost/",
                                               scope="user-top-read"))

results = sp.current_user_saved_tracks()

for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])



# multiple users 
from pprint import pprint

while True:
    username = input("Type the Spotify user ID to use: ")
    token = util.prompt_for_user_token(username, show_dialog=True)
    sp = spotipy.Spotify(token)
    pprint(sp.me())

TypeError: ignored

In [ ]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="7857e62ab9094421af53b8c63e70fad9",
                                               client_secret="425770791b664857be5af72dd0918737",
                                               redirect_uri="http://example.com",
                                               scope="user-library-read"))
sp.user('joshhall811')

Enter the URL you were redirected to: localhost:8888


SpotifyOauthError: ignored

In [ ]:
import requests
# authentication with Spotify; we need to get an access token 
# replace this with your client id and client secret; or using this is also fine
#client_id = '7857e62ab9094421af53b8c63e70fad9'
#client_secret = '425770791b664857be5af72dd0918737'
client_id = '5cb7cba382de499bbf9f7a1a19ad4fb6' #val's
client_secret = 'f70d0d7d856549ecaf367f17279742e1' #val's

import base64
messageBytes = f'{client_id}:{client_secret}'.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')

url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization':f'Basic {base64Message}'}
r = requests.post(url, headers = headers, data = {'grant_type':'client_credentials'})

In [ ]:
r.status_code

200

In [ ]:
import json

# this access token expires in an hour; so need to regenerate every hour
access_token = json.loads(r.text)['access_token']

In [ ]:
access_token

'BQAULI6Cid7A5XeX5O0UKi3hweZ1D02N3kk1JF46SCtau_k294s6UhGJsMbhZgdTgHYw7Mq2kkt2ymrap1E'

In [ ]:
# call the audio feature api 
# documentation: https://developer.spotify.com/console/get-audio-features-track/?id=06AKEBrKUckW0KREUWRnvT

track_id = '5OTH21tpmWn9YL3XzfB8AX'
audio_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
headers = {'Authorization':'Bearer ' + access_token}

r_audio = requests.get(audio_url, headers = headers)

In [ ]:
r_audio

<Response [200]>

In [ ]:
a = json.loads(r_audio.text)
print(a)

{'danceability': 0.709, 'energy': 0.643, 'key': 10, 'loudness': -5.324, 'mode': 1, 'speechiness': 0.0385, 'acousticness': 0.0504, 'instrumentalness': 3.68e-05, 'liveness': 0.283, 'valence': 0.666, 'tempo': 103.999, 'type': 'audio_features', 'id': '5OTH21tpmWn9YL3XzfB8AX', 'uri': 'spotify:track:5OTH21tpmWn9YL3XzfB8AX', 'track_href': 'https://api.spotify.com/v1/tracks/5OTH21tpmWn9YL3XzfB8AX', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5OTH21tpmWn9YL3XzfB8AX', 'duration_ms': 206007, 'time_signature': 4}


In [ ]:
r_audio.text

NameError: ignored

Get Recommendation API

In [ ]:
def get_rec(access_token, seed_artist_id, seed_genre_id, seed_track_id):
    # can adjust the limit; default returns 20 recommended tracks 
    rec_url = f'https://api.spotify.com/v1/recommendations?market=US&seed_artist={seed_artist_id}&seed_tracks={seed_track_id}'

    headers = {'Authorization':'Bearer ' + access_token}

    r_rec = requests.get(rec_url, headers = headers)
    rec_text = json.loads(r_rec.text)

    track_names = []
    track_ids = []
    for track in rec_text['tracks']:
        track_names.append(track['name'])
        track_ids.append(track['id'])
    return list(zip(track_ids, track_names))

In [ ]:
get_rec(access_token, '4S5MLjwRSi0NJ5nikflYnZ', '', '66U0ASk1VHZsqIkpMjKX3B')
# need to call the get audio features API to get their audio data  

[('5nIrdpo8eXQU3YgZelEXkd', 'Shots & Squats'),
 ('2BrEHx5ITLJHYF7A68XAiE', 'Ok'),
 ('37neP54nTnm3pVmJEX3B35', 'Grillkrydda'),
 ('3UkPUZcqN1x41vTX6AQ8rA', 'Give Me Your Money'),
 ('4G5tyjccKnH2yGgh9TgPgy', 'Over and Out (feat. Charlott Boss)'),
 ('3bWZoAaUDrLk0yLthtKSbl', 'Magic Symphony'),
 ('3vYP0NDCcoPiUHpmtq9YIq', 'Trombone'),
 ('1IDXuroj7pqZzrentmgh4H',
  'My Life Is a Party - Ryan T. & Rick M. Radio Edit'),
 ('385h9lN4b8sw2qmxpAvTAQ', 'Heart Starts to Beat'),
 ('7o0z4V1GatJJebIuPjVnpz', 'Jamrock Land'),
 ('0wi0Hn8puUPmYdZ0JvpG2H', 'Hide and Seek'),
 ('2j3sJgcBJ0qMSMAr2t12v8', "Don't U Worry (feat. Barbara Moleko)"),
 ('0rqkWGtTnfK7ZAbesB7HsY', 'Dying'),
 ('6bkzn3e9gtiP0xJTJnQ2eM', 'I`m OK'),
 ('6rF1rDhLbjgNGq5UOfqIJi', 'Streets Ahead 2015 (Out of Control)'),
 ('0DBTrH6kBbF8CzNVcu5MQR', 'Pong Dance'),
 ('3kf0ERnVuqQs62SIpj4HN8', 'Gam Gam'),
 ('1GL0EEkKCUT2yK52sjWkOt', 'Bailar'),
 ('2gCF9TZUNUnUXH9YWQS7Ea', 'Wicked Wonderland - Extended Mix'),
 ('1ucAgkQ4EQDCSXQ2kEk99k', 'UNO')]

In [ ]:
#get users top tracks
#set to 50 tracks, can change for better assignment to clusters for the users
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="7857e62ab9094421af53b8c63e70fad9",
                                               client_secret="425770791b664857be5af72dd0918737",
                                               redirect_uri="http://localhost",
                                               scope="user-top-read"))#need user-top-read scope for this, can add others like playlist creation
tracks = sp.current_user_top_tracks(50) #dictionary of top tracks

#create list of ids of top tracks
ids = []
for track in tracks["items"]:
  ids.append(track["id"])

#can now use ids to get attributes or other data as needed for cluster assingment for the users




<Response [404]>
